# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [154]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3
import sqlalchemy

import re
import nltk
nltk.download(['punkt', 'wordnet','stopwords'])


from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords


[nltk_data] Downloading package punkt to /home/ania/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ania/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ania/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [155]:
# load data from database
sql_engine = create_engine('sqlite:///DisasterResponse.db',echo=False)
connection = sql_engine.raw_connection()  #had to have this line otherwise froze

table_name = str(sql_engine.table_names()[0])
print(table_name)

df = pd.read_sql("SELECT * FROM '{}'".format(table_name),con=connection)
#X = 
#Y = 

data


In [156]:
sql_engine.table_names()

['data']

In [157]:
df.shape

(26216, 40)

In [158]:
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [159]:
df.isnull().sum()

id                            0
message                       0
original                  16046
genre                         0
related                       0
request                       0
offer                         0
aid_related                   0
medical_help                  0
medical_products              0
search_and_rescue             0
security                      0
military                      0
child_alone                   0
water                         0
food                          0
shelter                       0
clothing                      0
money                         0
missing_people                0
refugees                      0
death                         0
other_aid                     0
infrastructure_related        0
transport                     0
buildings                     0
electricity                   0
tools                         0
hospitals                     0
shops                         0
aid_centers                   0
other_in

In [160]:
# import matplotlib.pyplot as plt
# plt.hist(df['index'])

In [161]:
df_twitter = pd.read_csv('Twitter-sentiment-self-drive-DFE.csv')

In [162]:
df_twitter.head()

,Unnamed: 0,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,sentiment,sentiment:confidence,our_id,sentiment_gold,sentiment_gold_reason,text
0,0,724227031,True,golden,236,NaN,5,0.7579,10001,5\n4,Author is excited about the development of the...,Two places I'd invest all my money if I could:...
1,1,724227032,True,golden,231,NaN,5,0.8775,10002,5\n4,Author is excited that driverless cars will be...,Awesome! Google driverless cars will help the ...
2,2,724227033,True,golden,233,NaN,2,0.6805,10003,2\n1,The author is skeptical of the safety and reli...,If Google maps can't keep up with road constru...
3,3,724227034,True,golden,240,NaN,2,0.8820,10004,2\n1,The author is skeptical of the project's value.,Autonomous cars seem way overhyped given the t...
4,4,724227035,True,golden,240,NaN,3,1.0000,10005,3,Author is making an observation without expres...,Just saw Google self-driving car on I-34. It w...


In [163]:
df_twitter.shape

(7156, 12)

### 2. Write a tokenization function to process your text data

In [165]:
print(stopwords.words('english'))
stopword_list = stopwords.words('english')

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [172]:

def tokenize(text):
    # get list of all urls using regex
    detected_urls = re.findall(url_regex,text) #all cases of pattern save in string for each case
    detected_emails = re.findall(emails_regex,text)
    detected_emails = [email.split()[0] for email in detected_emails]
    detected_ips = re.findall(ips_regex,text)
    
#     if(len(detected_urls)>0 or len(detected_emails)>0 or len(detected_ips)>0):
        
#         print('***',detected_urls,detected_emails,detected_ips)
    
    
    ##     !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!add also stop words, noun and verb lemmatization
    
    # deal with RT and make category of genre in pipeline
    
    pattern = re.compile(r'[^a-zA-Z0-9]')
    stopword_list = stopwords.words('english')
    
#    replace each url/email/ips in text string with placeholder
    for url in detected_urls:
        
        text = re.sub(url,'urlplaceholder',text)   
                 

    for email in detected_emails:
        
        text = re.sub(email,'emailplaceholder',text)
            

    for ip in detected_ips:
        
        text = re.sub(ip,'ipplaceholder',text)       
        
            
    text = text.lower()  
    print(text)
    for stop_word in stopword_list:
        
        if(stop_word in text):
             text.replace(stop_word,'')
        #print(text)
    #print(text)
    
    text = re.sub(pattern,' ',text)
    
    tokens = word_tokenize(text.lower())
    
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:

        if(tok not in stopword_list):      
    
            clean_tok = lemmatizer.lemmatize(lemmatizer.lemmatize(tok.strip()),pos='v')
            clean_tokens.append(clean_tok)

    return clean_tokens

In [173]:
for message in df.message:
    print(tokenize(message))

weather update - a cold front from cuba that could pass over haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']
is the hurricane over or is it not over
['hurricane']
looking for someone but no name
['look', 'someone', 'name']
un reports leogane 80-90 destroyed. only hospital st. croix functioning. needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroy', 'hospital', 'st', 'croix', 'function', 'need', 'supply', 'desperately']
says: west side of haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight']
information about the national palace-
['information', 'national', 'palace']
storm at sacred heart of jesus
['storm', 'sacred', 'heart', 'jesus']
please, we need tents and water. we are in silo, thank you!
['please', 'need', 'tent', 'water', 'silo', 'thank']
i would like to receive the messages, thank you
['would', 'like', 'receive', 'message', 'thank']
i am in croix-des-bouquets. we 

we are in galet, we are hungry - tabarre 41 a little further up from the american embassy.
['galet', 'hungry', 'tabarre', '41', 'little', 'american', 'embassy']
we have a commitee from santo 15 nameajeans: youth association in action for a new society that was founded in september 2009, the president is..
['commitee', 'santo', '15', 'nameajeans', 'youth', 'association', 'action', 'new', 'society', 'found', 'september', '2009', 'president']
there are a lot of criminals in jacmel city. we are asking the police to come there.
['lot', 'criminal', 'jacmel', 'city', 'ask', 'police', 'come']
we are dying of hunger in petion ville
['die', 'hunger', 'petion', 'ville']
i'm exactly at akay in an area they name toman 43 miles near by kabar in a location name kafou lachomy.
['exactly', 'akay', 'area', 'name', 'toman', '43', 'mile', 'near', 'kabar', 'location', 'name', 'kafou', 'lachomy']
what time will the american embassy open tomorrow?
['time', 'american', 'embassy', 'open', 'tomorrow']
we are a 

aid should reach the victims outside the city of p-au-p. we are in gonaives when u get this message
['aid', 'reach', 'victim', 'outside', 'city', 'p', 'au', 'p', 'gonaives', 'u', 'get', 'message']
sos water and food. need to find where this adress is located. ..
['so', 'water', 'food', 'need', 'find', 'adress', 'locate']
we need help please. .. call us
['need', 'help', 'please', 'call', 'u']
we haven't gotten help like water and food. chiefs of the national television
['get', 'help', 'like', 'water', 'food', 'chief', 'national', 'television']
overpopulation in the city of cay. there is no security in this city. the cost of goods is risings. the people of cayes are overwhelmed.
['overpopulation', 'city', 'cay', 'security', 'city', 'cost', 'good', 'rise', 'people', 'cayes', 'overwhelm']
.. for the vehicules that must bring back limbeense and bas-limbeense. thanks for helping them. notes.. follow up from other message sebastien
['vehicules', 'must', 'bring', 'back', 'limbeense', 'ba', 'li

there's a problem with the signal which blocks transactions
['problem', 'signal', 'block', 'transaction']
good morning, could you please tell me where i can find a western union office that's open in the capital?
['good', 'morning', 'could', 'please', 'tell', 'find', 'western', 'union', 'office', 'open', 'capital']
we are at the immaculate conception hospital in les cayes, the complete adress is in rue duvivier - thanks a lot, may god bless you.
['immaculate', 'conception', 'hospital', 'le', 'cayes', 'complete', 'adress', 'rue', 'duvivier', 'thank', 'lot', 'may', 'god', 'bless']
good day digicel give me a chance so i ca call someone please
['good', 'day', 'digicel', 'give', 'chance', 'ca', 'call', 'someone', 'please']
good day! sweetheart i love you forever baby
['good', 'day', 'sweetheart', 'love', 'forever', 'baby']
the association of the youth in bon repos for the development of mole st nicolas are the market trying to find gasoline, the gallon for  50
['association', 'youth', 'bon'

graphic designer - it professional.
['graphic', 'designer', 'professional']
please i am trapped in a restaurant in rue du centre, port-au-prince. please send help, please.
['please', 'trap', 'restaurant', 'rue', 'du', 'centre', 'port', 'au', 'prince', 'please', 'send', 'help', 'please']
no one ever came to delmas 30, we have no water to drink, a lot of people have died, the bodies smell. thank you.
['one', 'ever', 'come', 'delmas', '30', 'water', 'drink', 'lot', 'people', 'die', 'body', 'smell', 'thank']
good morning, we heard about you. we live in canape vert in saint-hilaire. we need help, we never got any.
['good', 'morning', 'hear', 'live', 'canape', 'vert', 'saint', 'hilaire', 'need', 'help', 'never', 'get']
we need help in carrefour, the area is devastated and we have a lot of problem since the quake
['need', 'help', 'carrefour', 'area', 'devastate', 'lot', 'problem', 'since', 'quake']
we need help in port de paix because all people who left port au prince came n port de paix kno

...

on my way to santiago with the team . photoshot mode on . ;) en pistaaaaaaaa
['way', 'santiago', 'team', 'photoshot', 'mode', 'en', 'pistaaaaaaaa']
having breakfast before going to concepcion. a truck is coming full of food, the church is being generous to the church.
['breakfast', 'go', 'concepcion', 'truck', 'come', 'full', 'food', 'church', 'generous', 'church']
rt amtimyap: @kc_concepcion sometimes love doesn't have to be complicated. for as long as you want to make each other better--sometime ..
['rt', 'amtimyap', 'kc', 'concepcion', 'sometimes', 'love', 'complicate', 'long', 'want', 'make', 'better', 'sometime']
rt ytimeschile: four or five giant waves following the earthquake in chile almost wipe out coastal towns near concepcion urlplaceholder ..
['rt', 'ytimeschile', 'four', 'five', 'giant', 'wave', 'follow', 'earthquake', 'chile', 'almost', 'wipe', 'coastal', 'town', 'near', 'concepcion', 'urlplaceholder']
estudio:interseismic strain accumulation measured by gps in the seismi

i saw it in santiago, chile. here's pic of the haunted house urlplaceholder
['saw', 'santiago', 'chile', 'pic', 'haunt', 'house', 'urlplaceholder']
big aftershocks today in chile. santiago seems to be pretty much unharmed by everything. very curious if i will see some damage in santiago.
['big', 'aftershock', 'today', 'chile', 'santiago', 'seem', 'pretty', 'much', 'unharmed', 'everything', 'curious', 'see', 'damage', 'santiago']
utsatte turen til santiago til i morgen, da broforbindelsen ble stengt for privattrafikk. forst‚àöelse for at hjelpearbeid f‚àör prioritet.
['utsatte', 'turen', 'til', 'santiago', 'til', 'morgen', 'da', 'broforbindelsen', 'ble', 'stengt', 'privattrafikk', 'forst', 'else', 'hjelpearbeid', 'f', 'r', 'prioritet']
everybody back to the city!! santiago. se nos va el veranito! urlplaceholder
['everybody', 'back', 'city', 'santiago', 'se', 'no', 'va', 'el', 'veranito', 'urlplaceholder']
chile: there was a 8.8 magnitude earthquake (feb. 27) with an epicenter 70 miles f

rt fuenzalida: rt @mikeportnoydt: well,the santiago show is on! however, its been moved to friday..it's out of our hands, but at leas ..
['rt', 'fuenzalida', 'rt', 'mikeportnoydt', 'well', 'santiago', 'show', 'however', 'move', 'friday', 'hand', 'lea']
visit to dichato succesful! urlplaceholder
['visit', 'dichato', 'succesful', 'urlplaceholder']
crudess any clue on how you guys will spend the \
['crude', 'clue', 'guy', 'spend']
omg!! we have a hooters on santo domingo, they should bring one to santiago!!!
['omg', 'hooter', 'santo', 'domingo', 'bring', 'one', 'santiago']
chile: concepcion was moved ten feet by the earthquake urlplaceholder
['chile', 'concepcion', 'move', 'ten', 'foot', 'earthquake', 'urlplaceholder']
urlplaceholder - leaving santiago to head to south of chile with help!
['urlplaceholder', 'leave', 'santiago', 'head', 'south', 'chile', 'help']
urlplaceholder - leaving santiago to head to south of chile with help!
['urlplaceholder', 'leave', 'santiago', 'head', 'south', '

morning to all, starting now return to santiago, hope way back less shaky than yesterday, want to be with my family. sent from blackberry
['morning', 'start', 'return', 'santiago', 'hope', 'way', 'back', 'le', 'shaky', 'yesterday', 'want', 'family', 'send', 'blackberry']
coming to #chile #santiago?don't want to spend + than us17/day?looking for a quiet #hostel? visit hostal de la barra urlplaceholder
['come', 'chile', 'santiago', 'want', 'spend', 'us17', 'day', 'look', 'quiet', 'hostel', 'visit', 'hostal', 'de', 'la', 'barra', 'urlplaceholder']
haats_pof it is. even im in santiago all i can think about is the people in the south
['haats', 'pof', 'even', 'im', 'santiago', 'think', 'people', 'south']
more and more #haitians in #santiago d.r.. common now to overhear different #creole conversations in the aisles of any supermarket.
['haitian', 'santiago', 'r', 'common', 'overhear', 'different', 'creole', 'conversation', 'aisle', 'supermarket']
stoping in talca for lunch, i've seen, pain, s

error: unbalanced parenthesis at position 20

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.